In [ ]:
pip install tensorflow_decision_forests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 113.4 MB/s eta 0:00:00


In [ ]:
pip install -U pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210079 sha256=7e141d495c8ace3f2eb94c3d0289702b308f1ec1f4edcfc3f65dd2625b3870e6
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [ ]:
rs = 5138
np.random.seed(rs)

In [ ]:
data = pd.read_csv('/content/td2.csv')
data.head(2)

,CaO,SiO,Na2O,K2O,MgO,Al2O3,Total Binder,H20,FA,CA,Alk,DRCM,Strength
0,149.124553,128.238074,0.629026,6.970144,7.063509,33.132388,359.649123,205.00,743.146756,975.00,30.613401,15.646735,13.160891
1,156.638687,144.570008,0.541351,5.499724,6.527574,40.671237,392.280000,196.14,725.350000,1110.16,29.619016,1.540000,30.000000


In [ ]:
data.describe()

,CaO,SiO,Na2O,K2O,MgO,Al2O3,Total Binder,H20,FA,CA,Alk,DRCM,Strength
count,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000
mean,301.682749,128.802555,0.997752,4.525785,13.915139,28.050344,520.754617,210.304233,671.605138,1076.788076,28.576207,5.746008,41.001930
std,74.870291,29.720274,0.317345,2.147598,4.442618,10.848318,81.546565,10.130957,57.667906,58.337505,7.479219,3.737616,10.598291
min,149.124553,61.373404,0.276570,1.937867,6.527574,10.425149,359.649123,196.140000,539.469711,965.243367,16.789435,1.100000,13.160891
25%,239.343796,109.299430,0.746904,3.015384,10.376583,18.954931,448.320000,205.000000,642.218270,1100.403367,22.456796,3.194573,33.419508
50%,308.600950,124.813016,1.031136,3.342505,13.417220,25.777132,529.310000,211.187837,672.160110,1110.160000,28.081271,4.520000,43.700000
75%,368.475156,149.868113,1.291728,6.112172,17.251989,33.826276,585.714286,223.306339,720.413212,1110.195000,33.531052,8.054085,48.557104
max,420.110509,206.529816,1.473173,10.886291,24.209246,58.102272,640.510000,225.030661,763.246745,1113.352123,47.602404,16.731437,56.528980


In [ ]:
data1 = data[['CaO', 'SiO', 'Total Binder', 'H20','FA', 'CA', 'Alk', 'DRCM']]
data2 = data[['CaO', 'SiO', 'Total Binder', 'H20','FA', 'CA', 'Alk','Strength']]
features = ['CaO', 'SiO', 'Total Binder', 'H20','FA', 'CA', 'Alk']
n_f = len(features)

In [ ]:
def f_val_min_max(data, features:list, change_pc=0):
  n_f = len(features)
  min_reduce = 1-change_pc/100
  max_increase = 1+change_pc/100
  min_vals = []
  max_vals = []
  for i in range(n_f):
    min_vals.append(min_reduce*min(data[features[i]]))
    max_vals.append(max_increase*max(data[features[i]]))

  return min_vals, max_vals

In [ ]:
def split_dataset(dataset, test_ratio=0.20):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train1, test1 = split_dataset(data1)
train2, test2 = split_dataset(data2)

In [ ]:
D_train = tfdf.keras.pd_dataframe_to_tf_dataset(train1, label="DRCM", task = tfdf.keras.Task.REGRESSION)
D_test = tfdf.keras.pd_dataframe_to_tf_dataset(test1, label="DRCM", task = tfdf.keras.Task.REGRESSION)

S_train = tfdf.keras.pd_dataframe_to_tf_dataset(train2, label="Strength", task = tfdf.keras.Task.REGRESSION)
S_test = tfdf.keras.pd_dataframe_to_tf_dataset(test2, label="Strength", task = tfdf.keras.Task.REGRESSION)

In [ ]:
tunerd = tfdf.tuner.RandomSearch(num_trials=int((51+38)), use_predefined_hps=True, trial_num_threads=10)  #*((3*8)/(5*1))
tuners = tfdf.tuner.RandomSearch(num_trials=int((51+38)*((1+8)/(5+3))), use_predefined_hps=True, trial_num_threads=10)

Diffusion Coefficient

In [ ]:
rf1t = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION, random_seed = rs, tuner = tunerd, verbose=0)
rf1t.compile()
rf1t.fit(D_train)
rf1t.compile(metrics=['mse'])
e = rf1t.evaluate(D_test, return_dict=True)
print(e)
print(e['mse'])
print(np.sqrt(e['mse']))

r = rf1t.predict(D_test)
s = np.array(test1["DRCM"])
l = []
for i in r:
  l.append(float(i))

abs(s - np.array(l))/s*100

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 480ms/step - loss: 0.0000e+00 - mse: 4.1992
{'loss': 0.0, 'mse': 4.199164390563965}
4.199164390563965
2.049186275223403
1/1 [==============================] - 0s 53ms/step


array([  5.00237544,  45.47957701,   1.14435635,  20.16118164,
       113.17228955,   9.39855851,  42.58792627,  22.41266651,
        72.41186997,  14.82868053,  28.94047328,  14.81790902,
        28.13123597,  11.37412715,   2.26313321,   0.37638986,
         0.67102524,   0.54632116,   5.36469962,   0.51143308,
         4.96661194,   3.61705498,   8.86762095,   7.56769919,
        12.69879246])

Strength

In [ ]:
rf2t = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION, random_seed = rs, tuner = tuners, verbose=0)
rf2t.fit(S_train)
rf2t.compile(metrics=['mse'])
e = rf2t.evaluate(S_test, return_dict=True)
print(e)
print(e['mse'])
print(np.sqrt(e['mse']))

r = rf2t.predict(S_test)
s = np.array(test2["Strength"])
l = []
for i in r:
  l.append(float(i))

abs(s - np.array(l))/s*100

1/1 [==============================] - 0s 185ms/step - loss: 0.0000e+00 - mse: 10.5837


{'loss': 0.0, 'mse': 10.583653450012207}
10.583653450012207
3.2532527491746177
1/1 [==============================] - 0s 57ms/step


array([ 1.29631084, 37.25018897,  0.31284524, 15.82741482,  3.78521275,
        0.5044012 , 16.48606384,  0.12980828,  6.85702068, 15.51015333,
       14.44165088, 12.57647939,  0.23965259,  1.60077044,  0.52919232,
        1.56240939,  1.54169163,  0.88678847,  0.11848418])

Find design

In [ ]:
def dif_cof(model, X):
  tp = pd.DataFrame(X, columns = features)
  t = tfdf.keras.pd_dataframe_to_tf_dataset(tp)
  r = model.predict(t)
  return r

def strength(model, X):
  tp = pd.DataFrame(X, columns = features)
  t = tfdf.keras.pd_dataframe_to_tf_dataset(tp)
  r = model.predict(t)
  return r

In [ ]:
X = [3,0,0,0,0,0,0]

In [ ]:
strength(rf2t, [X])

1/1 [==============================] - 0s 199ms/step


array([[36.698322]], dtype=float32)

In [ ]:
def mass_of_material(M_CaO, M_SiO2, M_Binder, fa=True, s=True, cond_min=False):

  if fa==False:
    mass = [M_CaO, M_SiO2]
    Cmax = [[0.673299, 0.490982],[0.490982, 0.313951]]   #Cement, Slag
  elif s==False:
    mass = [M_CaO, M_SiO2]
    Cmax = [[0.673299, 0.026649],[0.170648, 0.635439]]   #Cement, Flyash
  else:
    mass = [M_CaO, M_SiO2, M_Binder]
    Cmax = [[0.673299, 0.026649, 0.490982],[0.170648, 0.635439, 0.313951], [1, 1, 1]]   #Cement, Flyash, Slag


  mass_m = np.linalg.solve(Cmax, mass)
  return mass_m

In [ ]:
def alkalinity(M_CaO, M_SiO2, M_Binder, fa=True, s=True, cond_min=False):
  if fa==False:
    mses = mass_of_material(M_CaO, M_SiO2, M_Binder, fa, s, cond_min) #cement, slag
    alk = np.array([1.7686, 9.165])*mses[0] + np.array([4.4165, 15.1393])*mses[1]

  elif s==False:
    mses = mass_of_material(M_CaO, M_SiO2, M_Binder, fa, s, cond_min) #cement, flyash
    alk = np.array([1.7686, 9.165])*mses[0] + np.array([6.4208, 24.47])*mses[1]

  else:
    mses = mass_of_material(M_CaO, M_SiO2, M_Binder, fa, s, cond_min) #cement, flyash, slag
    alk = np.array([1.7686, 9.165])*mses[0] + np.array([6.4208, 24.47])*mses[1] + np.array([4.4165, 15.1393])*mses[2]

  return alk

In [ ]:
tdc = 8
tstr = 40

In [ ]:
from pymoo.core.problem import ElementwiseProblem

class functions(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=n_f,
                         n_obj=2,
                         n_ieq_constr=0,
                         #n_eq_constr=1,
                         #["Cao","SiO2","H20",'Alk', "Total Binder","FA","CA"]
                         ##['CaO', 'SiO', 'Total Binder', 'H20','FA', 'CA', 'Alk']
                         #xl=np.array([100, 50, 500, 205, 500, 950, 15,]),
                         #xu=np.array([400, 200, 650, 220, 900, 1200, 29])
                         xl = f_val_min_max(data, features, change_pc=2)[0],
                         xu = f_val_min_max(data, features, change_pc=2)[1]
                         )


    def _evaluate(self, x, out, *args, **kwargs):

        Xt = []

        for i in range(n_f):
          Xt.append(x[i])

        f1 = abs(dif_cof(rf1t, [Xt]) - tdc)
        f2 = abs(strength(rf2t, [Xt]) - tstr)

        #h1 = np.sum(mass_of_material(x[0], x[1], fa=True, s=False, cond_min=False)) - x[2]

        out["F"] = [f1, f2]
        #out["H"] = [h1]

problem = functions()

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling

algorithm = NSGA2(
    pop_size=75,
    n_offsprings=30,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.7, eta=5),
    mutation=PM(eta=7),
    eliminate_duplicates=True
)

from pymoo.termination import get_termination

termination = get_termination("n_gen", 20)

In [ ]:
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=rs,
               save_history=True,
               verbose=True)

X = res.X
F = res.F

1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 51ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 45ms/step


...

1/1 [==============================] - 0s 71ms/step


1/1 [==============================] - 0s 85ms/step


1/1 [==============================] - 0s 83ms/step


1/1 [==============================] - 0s 66ms/step


1/1 [==============================] - 0s 77ms/step
    20 |      645 |      8 |  0.0081368607 |         ideal


In [ ]:
X

array([[ 371.20769949,  163.57084144,  640.35588351,  224.21949669,
         556.00577751, 1001.82281559,   31.30727446],
       [ 402.17303346,   71.35592211,  403.63524815,  221.36033846,
         583.83244598, 1101.87123544,   36.68534602],
       [ 427.38083226,   63.27924585,  419.36172901,  223.34284192,
         553.876199  , 1061.18459543,   16.97850623],
       [ 408.39572647,   80.66072792,  596.0596581 ,  225.94774191,
         557.47588789,  964.39486446,   30.53916541],
       [ 427.38083226,   68.2312471 ,  419.36172901,  223.34284192,
         577.59557319, 1061.18459543,   16.97850623],
       [ 427.38083226,   60.75981778,  419.36172901,  223.34284192,
         553.876199  , 1064.39216447,   17.18555305],
       [ 405.99144244,   69.68579704,  402.82629532,  222.33891819,
         538.74827801, 1099.5603324 ,   24.68395813],
       [ 356.17842523,   69.13755991,  590.60220061,  215.88679799,
         676.98706014,  970.31147864,   38.59512098]])

In [ ]:
F

array([[1.21076584e+00, 1.72119141e-02],
       [3.67319584e-01, 1.95274353e-02],
       [1.12948418e-01, 1.04598999e-01],
       [2.07783508e+00, 2.81143188e-03],
       [2.46620178e-03, 2.84137726e-01],
       [1.35306358e-01, 7.36389160e-02],
       [1.35591030e-01, 2.82249451e-02],
       [2.52496433e+00, 5.03540039e-04]])

In [ ]:
def dif_cof(model, X):
  tp = pd.DataFrame(X, columns = features)
  t = tfdf.keras.pd_dataframe_to_tf_dataset(tp)
  r = model.predict(t)
  return r

def strength(model, X):
  tp = pd.DataFrame(X, columns = features)
  t = tfdf.keras.pd_dataframe_to_tf_dataset(tp)
  r = model.predict(t)
  return r

In [ ]:
def dist_cof(model1, model2, X):
  tp = pd.DataFrame(X, columns = features)
  t = tfdf.keras.pd_dataframe_to_tf_dataset(tp)
  r1 = model1.predict(t)
  r2 = model2.predict(t)
  r = list(np.sqrt((1-r1/tdc)**2 + (1-r2/tstr)**2))
  rmin_i = r.index(min(r))

  return X[rmin_i]

In [ ]:
def dist(F):
  n=len(F)
  f = list(F)
  err = 5138
  erl = []
  for i in range(n):
    err1 = f[i][1]
    if err1<=err and f[i][0]>0.5:
      err = err1
    else:
      err = err
    erl.append(err)

  rmin = erl.index(err)
  return rmin

In [ ]:
X[int(dist(F))]

array([356.17842523,  69.13755991, 590.60220061, 215.88679799,
       676.98706014, 970.31147864,  38.59512098])

In [ ]:
v = (X[dist(F)])

In [ ]:
#v = dist_cof(rf1t, rf2t, X)
v2 = []
for i in v:
  v2.append(int(i))

In [ ]:
v2

[356, 69, 590, 215, 676, 970, 38]

In [ ]:
dif_cof(rf1t, [list(v)])

1/1 [==============================] - 0s 85ms/step


array([[10.524964]], dtype=float32)

In [ ]:
strength(rf2t, [list(v)])

1/1 [==============================] - 0s 63ms/step


array([[40.000504]], dtype=float32)

In [ ]:
a = [256.247633, 108.052433, 435.87, 196.1415, 692.93, 1110.16, 24.82823766]
b = [257.93277269,  171.9759386 ,  437.97998386,  201.69973688, 626.68504044, 1117.74710378,   48.14336098]

In [ ]:
dif_cof(rf1t, [list(b)])

1/1 [==============================] - 0s 69ms/step


array([[2.9048355]], dtype=float32)

In [ ]:
strength(rf2t, [list(b)])

1/1 [==============================] - 0s 73ms/step


array([[41.192986]], dtype=float32)